![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=Mathematics/StatisticalReasoning/statistical-reasoning.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Introduction

In this notebook we are going to explore statistics.

[Jerry Sokoloski](https://en.wikipedia.org/wiki/Jerry_Sokoloski) is a Canadian actor and former basketball player. At 7'4" (2.24 m) he is also Canada's tallest man.

![Jerry](images/jerry.jpg)

*Jerry Sokoloski walking down the street in Toronto. Photo credit: [STAN BEHAL/Toronto Sun](https://torontosun.com/author/stanbehal).*

Ok, so 7'4" seems really tall, right? What if we want to find out how Jerry *measures up* to the rest of the Canadian population of adult men? For that task, we're going to need some more information. We can find out just how out*stand*ing Jerry is by comparing his height to thousands of other men's heights in the United States, which we'll assume should be about the same as Canada.

We'll use Python to load in some data made available by the 2021 Center for Disease Control (CDC) National Health Interview Survey. Click on the code cell below, then click the `▶Run` button to download and visualize the data.

In [ ]:
import requests
from io import BytesIO
from zipfile import ZipFile
import pandas as pd
import plotly.express as px

try:
    r = requests.get('https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Datasets/NHIS/2021/adult21csv.zip')
    with ZipFile(BytesIO(r.content)) as z:
        f = z.extractall()
    df = pd.read_csv('adult21.csv')
except:
    df = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/Mathematics/StatisticalReasoning/data/adult21.csv')

px.histogram(df['HEIGHTTC_A'], title='Adult Heights', labels={'value':'Height (inches)'}).update_layout(showlegend=False)

This **histogram** of height data, in inches, shows how many individuals in the data set were *count*ed with that height. We can see that there are some very tall individuals with heights of 96 to 99 inches (2.4 to 2.5 m). According to the [NHIS Codebook](https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Dataset_Documentation/NHIS/2021/adult-codebook.pdf), those are not actual height measurments.

|Height Code|Description|
|-|-|
|96|extreme value or sex unknown|
|97|refused|
|98|not ascertained|
|99|don't know|

So we can remove those values and make a new histogram.

In [ ]:
df.drop(df[df['HEIGHTTC_A'] > 95].index, inplace=True)
px.histogram(df['HEIGHTTC_A'], title='Adult Heights', labels={'value':'Height (inches)'}).update_layout(showlegend=False)

Since we want to compare Jerry Sokoloski's height to adult male heights, let's use just the rows where `SEX_A` is `1` (male).

In [ ]:
mdf = df[df['SEX_A'] == 1]
px.histogram(mdf['HEIGHTTC_A'], title='Male Adult Heights', labels={'value':'Height (inches)'}).update_layout(showlegend=False)

For comparison, we can compare that to the female heights in the data set.

In [ ]:
px.histogram(df[df['SEX_A']==2]['HEIGHTTC_A'], title='Female Adult Heights', labels={'value':'Height (inches)'}).update_layout(showlegend=False)

## Normal Distribution

When a histogram, like these, has a large bump in the middle and smaller values ('tails') at either end, we say that the dataset is approximately **normally** distributed.

You may have heard the term [bell curve](https://en.wikipedia.org/wiki/Normal_distribution), that is *the* normal distribution. It's an idealized model for any normally distributed dataset.

## The Mean of a Distribution

The big bump in the middle of the data is the **mean** of the distribution. The mean of a normally distributed dataset is usually very close in value to the middle element. It's the average of all of the elements in the dataset. Usually, we use $\mu$ as a symbol for the mean.

### Finding the Mean Mathematically

If you want to calculate the mean for a dataset, you need to add up all of the elements and then divide by the number of elements. So, if we have the dataset ${1,2,3,4,5}$, which has 5 elements, the mean is given by

$\mu = \frac{1+2+3+4+5}{5} = \frac{15}{5} = 3$

In general, if we have the dataset ${x_1, x_2, x_3, \ldots, x_N}$, where $N$ is the number of elements, then the mean is given by

$\mu = \frac{x_1 + x_2 + x_3 + \ldots + x_N}{N} = \frac{\sum_i x_i}{N}$

Notice that we use the symbol $\sum$ as a shortcut to writing the summation of every element in the dataset.

Let's find the mean of our height data.

In [ ]:
mdf['HEIGHTTC_A'].mean()

### The Median and the Mode of a Distribution

There are two others numbers that can help us describe the distribution of a dataset. The first is the **median**, which is the **middle** number of a dataset. The **mode** is the most frequently occuring value in the dataset.

In [ ]:
mdf['HEIGHTTC_A'].median()

In [ ]:
mdf['HEIGHTTC_A'].mode()

In a normally distributed dataset, the median, mean, and mode should all be approximately equal.

## The Shape of a Distribution: Skewness

Sometimes there are elements in a dataset that can change the shape of the histogram and its underlying distribution. Let's use the heights dataset as an example. Suppose we included the heights of NBA basketball players in the dataset. This would definitely make the value of the mean higher, and might change the value of the median. Similarly, if we including the heights of children in the dataset, the mean would be lower and the median might change.

Let's plot the histogram of the heights dataset again and see what the shape looks like when children or NBA players are included.

In [ ]:
try:
    r = requests.get('https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Datasets/NHIS/2018/samchildcsv.zip')
    with ZipFile(BytesIO(r.content)) as z:
        f = z.extractall()
    cdf = pd.read_csv('samchild.csv')
except:
    cdf = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/Mathematics/StatisticalReasoning/data/samchild.csv')
cdf.drop(cdf[cdf['CHGHT_TC'] > 95].index, inplace=True)
mcdf = cdf[cdf['SEX'] == 1]
px.histogram(pd.concat([mdf['HEIGHTTC_A'], mcdf['CHGHT_TC']]), title='Male Adult and Child Heights', labels={'value':'Height (inches)'}).update_layout(showlegend=False)

In [ ]:
# Get data from basketball-reference.com
'''
import requests
from bs4 import BeautifulSoup
r  = requests.get('http://www.basketball-reference.com/leagues/NBA_2021_totals.html')
soup = BeautifulSoup(r.text)
heights = {}
for link in soup.findAll('table')[0].findAll('a'):
    if '/players' in link.get('href'):
        url = 'http://www.basketball-reference.com'+link.get('href')
        player_page = requests.get(url)
        soup2 = BeautifulSoup(player_page.text)
        name = soup2.find_all('title')[0].text.split(' Stats')[0]
        for paragraph in soup2.findAll('p'):
            if '-' in paragraph.text and 'lb' in paragraph.text:
                heightFeet = paragraph.text.split('-')[0]
                heightInches = paragraph.text.split('-')[1].split(',')[0]
                print(name, heightFeet + ' and ' + heightInches)
                heights[name] = int(heightFeet)*12 + int(heightInches)
                break
ndf = pd.DataFrame(heights.items(), columns=['Name', 'Height (inches)'])
'''

ndf = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/Mathematics/StatisticalReasoning/data/NBAheights2021.csv')
px.histogram(pd.concat([mdf['HEIGHTTC_A'], ndf['Height (inches)']]), title='Male Adult and NBA Player Heights', labels={'value':'Height (inches)'}).update_layout(showlegend=False)

We can see that including different sets of data skews the histogram. The following code cell will show all three histograms so we can compare them.

In [ ]:
import plotly.subplots as sp
fig = sp.make_subplots(1, 3, subplot_titles=['Male Adult Heights', 'Male Adult and Child Heights', 'Male Adult and NBA Player Heights'])
fig.append_trace(px.histogram(mdf['HEIGHTTC_A'])['data'][0], 1, 1)
fig.append_trace(px.histogram(pd.concat([mdf['HEIGHTTC_A'], mcdf['CHGHT_TC']]))['data'][0], 1, 2)
fig.append_trace(px.histogram(pd.concat([mdf['HEIGHTTC_A'], ndf['Height (inches)']]))['data'][0], 1, 3)
fig.update_layout(showlegend=False).update_xaxes(title_text='Height (inches)').show()

Adding in the children's heights made the big bump in the histogram shift to the right. The children's heights also caused the mean to decrease but the median to stay the same. When a distribution has its bump shifted to the right like this, we say that the distribution is **skewed left**, **left-skewed**, or **negatively skewed**.

When we added the NBA heights, the big bump in the middle shifted to the left. The NBA heights caused the mean to increase but the median to stay the same.  When a distribution has its bump shifted to the left like this, we say that the distribution is **skewed right**, **right-skewed**, or **positively skewed**.

Leaving out both the children's heights and the NBA heights, the distribution looks roughly **symmetrical**. This means that the distribution is not skewed left or right, but looks roughly as you look away from the middle bump in either direction.

In general:
* If the median is higher than the mean, the distribution is skewed right.
* If the mean is higher than the median, the distribution is skewed left.
* If the median and the mean are approximately equal, the distribution is symmetrical.

### Outliers

Sometimes, if there are only a few data elements at either end of the histogram, the data elements causing the skewness might be outliers. An **outlier** is a data element that could be an extremely rare observation or a mistake in the data the data. Jerry Sokoloski's height, like that of many NBA players, may be an outlier in a data set of adult male heights.

## The Shape of a Distribution: Standard Deviation

The mean, median, and mode are good indicators of how normal a distribution or dataset might be, but they don't tell us the whole story. If we want to know about how spread out the elements of a dataset are, then we need a new statistic. The standard deviation of a dataset, written as $\sigma$, is the measurement of how much the elements of the dataset vary as a group. In other words, how spread out the histogram is. For a normal distribution about 68% of the values should be within one standard deviation of the mean, about 95% of the data should be within $2 \sigma$ of the mean, and about 99.7% of the data should be within $3 \sigma$ of the mean. This is often called the **68-95-99.7 rule**.

![normal distribution](images/normaldist.png)

*Image by [John Canning](http://johncanning.net/wp/?p=1202)*

Using the standard deviation, we can also find out how far a single data element is from the rest of the dataset. We're going to use the standard deviation ($\sigma$) of the adult male heights dataset to find out how different Jerry Sokoloksi's height is. First, find the value of $\sigma$ for the heights dataset.

In [ ]:
import numpy as np
sigma = np.std(mdf['HEIGHTTC_A'])
mu = mdf['HEIGHTTC_A'].mean()

print('The standard deviation is {:.3} inches.'.format(sigma))
print('The mean is {:.3} inches.'.format(mdf['HEIGHTTC_A'].mean()))
print('About 68% of men will have a height between {:.3} inches and {:.3} inches.'.format(mu-sigma, mu+sigma))
print('About 95% of men will have a height between {:.3} inches and {:.3} inches.'.format(mu-2*sigma, mu+2*sigma))
print('About 99.7% of men will have a height between {:.3} inches and {:.3} inches.'.format(mu-3*sigma, mu+3*sigma))

Only about 0.3% of the population will have a height outside of $3\sigma$ from the mean, and only about 0.15% will have a height more than $3\sigma$ above the mean. So we can say that Jerry Sokoloski's height of 79 inches is in the top 0.15% of male heights in our data set.

# Conclusion

When given a dataset, we can check if it is normally distributed by visually checking its plotted **histogram** or calculating and comparing the **mean**, **median**, and **mode**.

The median and the mean can tell us if the distribution is **skewed** left or right, which may indicate **outliers** in the dataset. Examining the shape of the histogram can also tell us about the skewness of the underlying distribution, and may indicate that we have outliers in the dataset.

The **standard deviation** tells us about the spread of the data.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)